In this section the API selected will be used so that we load our ES db

we ll use docker for elastic search and fill the details in the .ENV file

- NYT-Times NewsWire API
- NYT-Books API
- GoodReads

// end points
// list of book categories : https://api.nytimes.com/svc/books/v3/lists/names.json
// number of reviewed book : https://api.nytimes.com/svc/books/v3/lists/best-sellers/history.json
// good reads : https://www.goodreads.com/book/review_counts.json
//

// need to establish the following :
//1. connection to the ES container
//
//2. loading the api request from the two api
//2. 1 for each API define loops and requirements


1. Connection to the container // Windows installation

1.Install Docker: Download and install Docker Desktop for Windows from the official Docker website. Follow the installation instructions specific to your Windows version.
2.Launch Docker: Once Docker is installed, launch Docker Desktop from the start menu or desktop shortcut. Make sure it's running before proceeding to the next steps.
3.Open a Command Prompt: Open the Command Prompt or PowerShell on your Windows machine.
4.Pull the Elasticsearch Docker Image: Use the following command to pull the official Elasticsearch Docker image from Docker Hub:

bash
Copy code
docker pull docker.elastic.co/elasticsearch/elasticsearch:8.8.1

5.Create a Docker Container
docker run -d --name elasticsearch_container -p 9200:9200 -p 9300:9300 -e "discovery.type=single-node" docker.elastic.co/elasticsearch/elasticsearch:8.8.1
docker run -d --name elasticsearch  -v /data/dump/:/usr/share/elasticsearch/data -p 9400:9400 -p 9500:9200 -e "discovery.type=single-node" elasticsearch:8.8.1
docker run -d --name elasticsearch761v2  -v /data/dump/:/usr/share/elasticsearch/data -p 9400:9400 -p 9500:9200 -e "discovery.type=single-node" elasticsearch:7.6.1

docker run --name elasticSearch -p 9200:9200 -p 9300:9300 -e "discovery.type=single-node" docker.elastic.co/elasticsearch/elasticsearch:7.5.2

6.Verify Elasticsearch Installation: Open your web browser and navigate to http://localhost:9200.


Once the connection is established we want to analyse in details the API and load them one by one without overloading the API from NYT or GoodReads

To interact with ES firstly you need to : pip install elasticsearch


In [8]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from dotenv import load_dotenv


es = Elasticsearch(hosts = "http://@localhost:9200")


index_name = 'nyt-newswire'
newswire_mapping = {
        'properties': {
            'abstract': {'type': 'text', 'analyzer': 'english'},
            'byline': {'type': 'text', 'analyzer': 'english'},
            'created_date': {'type': 'date'},
            'des_facet': {'type': 'keyword'},
            'first_published_date': {'type': 'date'},
            'geo_facet': {'type': 'keyword'},
            'item_type': {'type': 'keyword'},
            'kicker': {'type': 'text'},
            'material_type_facet': {'type': 'keyword'},
            'multimedia': {
                'type': 'nested',
                'properties': {
                    'caption': {'type': 'text', 'analyzer': 'english'},
                    'copyright': {'type': 'text', 'analyzer': 'english'},
                    'format': {'type': 'keyword'},
                    'height': {'type': 'integer'},
                    'subtype': {'type': 'keyword'},
                    'type': {'type': 'keyword'},
                    'url': {'type': 'text', 'analyzer': 'english'},
                    'width': {'type': 'integer'}
                }
            },
            'org_facet': {'type': 'keyword'},
            'per_facet': {'type': 'keyword'},
            'published_date': {'type': 'date'},
            'section': {'type': 'keyword'},
            'slug_name': {'type': 'keyword'},
            'source': {'type': 'text', 'analyzer': 'english'},
            'subheadline': {'type': 'text', 'analyzer': 'english'},
            'subsection': {'type': 'keyword'},
            'thumbnail_standard': {'type': 'text', 'analyzer': 'english'},
            'title': {'type': 'text'},
            'updated_date': {'type': 'date'},
            'uri': {'type': 'keyword'},
            'url': {'type': 'text', 'analyzer': 'english'}
        }
    }

newswire_settings = {
    "number_of_shards": 2,
    "number_of_replicas": 2
}

 # Create the index
response = es.indices.create(index=index_name, mappings=newswire_mapping, 
                             settings=newswire_settings)

# Check the response
  
if response['acknowledged']:
    print('Index created successfully.')
else:
    print('Failed to create index.')

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [nyt-newswire/rB0uZqZ-SUqNQZ0vKsRnbA] already exists')

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from dotenv import load_dotenv

##########################################################################
# we prepare the mapping for the new ESindex
##########################################################################
##########################################################################
# Mapping and index for yhr NYT - Book API 
##########################################################################
#To be fixed 


es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

index_name = 'NYT-booksList'
books_list_index_body = {
    'properties': {
        'amazon_product_url': {'type': 'keyword'},
        'asterisk': {'type': 'integer'},
        'bestsellers_date': {'type': 'date'},
        'book_details': {
            'type': 'nested',
            'properties': {
                'age_group': {'type': 'keyword'},
                'author': {'type': 'keyword'},
                'contributor': {'type': 'keyword'},
                'contributor_note': {'type': 'text', 'analyzer': 'english'},
                'description': {'type': 'text', 'analyzer': 'english'},
                'price': {'type': 'float'},
                'primary_isbn10': {'type': 'keyword'},
                'primary_isbn13': {'type': 'keyword'},
                'publisher': {'type': 'text', 'analyzer': 'english'},
                'title': {'type': 'text', 'analyzer': 'english'}
            }
        },
        'dagger': {'type': 'integer'},
        'display_name': {'type': 'keyword'},
        'isbns': {
            'type': 'nested',
            'properties': {
                'isbn10': {'type': 'keyword'},
                'isbn13': {'type': 'keyword'}
            }
        },
        'list_name': {'type': 'keyword'},
        'published_date': {'type': 'date'},
        'rank': {'type': 'integer'},
        'rank_last_week': {'type': 'integer'},
        'reviews': {
            'type': 'nested',
            'properties': {
                'article_chapter_link': {'type': 'keyword'},
                'book_review_link': {'type': 'keyword'},
                'first_chapter_link': {'type': 'keyword'},
                'sunday_review_link': {'type': 'keyword'}
            }
        },
        'weeks_on_list': {'type': 'integer'}
    }
}

book_list_settings = {
    "number_of_shards": 2,
    "number_of_replicas": 2
}


 # Create the index
response = es.indices.create(index=index_name, mappings=books_list_index_body,
                             settings=book_list_settings)

# Check the response
  
if response['acknowledged']:
    print('Index created successfully.')
else:
    print('Failed to create index.')

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from dotenv import load_dotenv

##########################################################################
# we prepare the mapping for the new ESindex
##########################################################################
##########################################################################
# Mapping and index for yhr NYT - Book API -- best sellers
##########################################################################
#To be fixed 


es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

index_name = 'NYT-books-best-sellers'
BS_books_index_body = {
    'properties': {
    'title': {'type': 'text', 'analyzer': 'english'},
    'description': {'type': 'text', 'analyzer': 'english'},
    'contributor': {'type': 'text', 'analyzer': 'english'},
    'author': {'type': 'text', 'analyzer': 'english'},
    'contributor_note': {'type': 'text', 'analyzer': 'english'},
    'price': {'type': 'float'},
    'age_group': {'type': 'keyword'},
    'publisher': {'type': 'text', 'analyzer': 'english'},
    'isbns': {
        'type': 'nested',
        'properties': {
            'isbn10': {'type': 'keyword'},
            'isbn13': {'type': 'keyword'},
        }
    },
    'ranks_history': {
        'type': 'nested',
        'properties': {
            'primary_isbn10': {'type': 'keyword'},
            'primary_isbn13': {'type': 'keyword'},
            'rank': {'type': 'integer'},
            'list_name': {'type': 'text', 'analyzer': 'english'},
            'display_name': {'type': 'text', 'analyzer': 'english'},
            'published_date': {'type': 'date'},
            'bestsellers_date': {'type': 'date'},
            'weeks_on_list': {'type': 'integer'},
            'ranks_last_week': {'type': 'integer', 'null_value': None},
            'asterisk': {'type': 'integer'},
            'dagger': {'type': 'integer'},
        }
    },
    'reviews': {
        'type': 'nested',
        'properties': {
            'book_review_link': {'type': 'keyword'},
            'first_chapter_link': {'type': 'keyword'},
            'sunday_review_link': {'type': 'keyword'},
            'article_chapter_link': {'type': 'keyword'},
            }
        },
    }
}

books_settings = {
    "number_of_shards": 2,
    "number_of_replicas": 2
}


 # Create the index
response = es.indices.create(index=index_name, mappings=BS_books_index_body, 
                             settings=books_settings)

# Check the response
  
if response['acknowledged']:
    print('Index created successfully.')
else:
    print('Failed to create index.')

The following are the requests and loading in the Elastic search DB


In [ ]:

from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from dotenv import load_dotenv
import os
import pprint
import requests
import time
import json
pp = pprint.PrettyPrinter(indent=4)
load_dotenv()
api_key = os.getenv("API_KEY")

########################################################
########################################################
# NEWSWIRE API
########################################################
########################################################


#Define the variables used in the API
#Get the list of sections needed to call the API Newswire
reqSections =  requests.get(f'https://api.nytimes.com/svc/news/v3/content/section-list.json?&api-key={api_key}')
sectionsList= [item['section'] for item in reqSections.json()['results']]
print(sectionsList)


########################################################
########################################################
# for tests ONLY ###
########################################################
sectionsList = sectionsList[:2]
print(sectionsList)
########################################################
########################################################


# iterate through the list 
for section in sectionsList:
     
    # Create a connection to Elasticsearch  

    #Request the Api
    content = requests.get(f'https://api.nytimes.com/svc/news/v3/content/all/{section}.json?&api-key={api_key}')
    #save into the ES DB
    res = content.json()
    pp.pprint(res) 
    docs = res['results']


    """

    # Prepare documents for indexing
    actions = [
        {
            '_index': index_name,
            '_source': doc
         }
        for doc in docs
    ]



    # Define an index and document type for ES
    index_name = 'NYT-newswire'

    # Bulk index the documents
    headers = {
        'Content-Type': 'application/json'
    }

    index_body = {
        'settings': {
            'number_of_shards': 1,
            'number_of_replicas': 0
        },
        'mappings': {
            'properties': {
                'field1': {'type': 'text'},
                'field2': {'type': 'keyword'}
            }
        }
    }




    #response = es.bulk(index=index_name, body=actions, headers=headers)


  
    # Check the response
    if response['result'] == 'created':
        print(f'{section} saved successfully')
    else:
        print('Failed to save content.')
    
    ######################################################
    time.sleep(2) ##### TO MODIFY ACCORDING API ALLOWANCE
    ######################################################
    """


In [ ]:

from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from dotenv import load_dotenv
import os
import pprint
import requests
import time
import json
pp = pprint.PrettyPrinter(indent=4)
load_dotenv()
api_key = os.getenv("API_KEY")

########################################################
########################################################
# BOOKS API
########################################################
########################################################


#Define the variables used in the API
#Get the list of sections needed to call the API Newswire
reqBookList = requests.get(f"https://api.nytimes.com/svc/books/v3/lists/names.json?api-key={api_key}")
booksList= [item['list_name'] for item in reqBookList.json()['results']]


########################################################
########################################################
# for tests ONLY ###
########################################################
booksList = booksList[:2]
print(booksList)
########################################################
########################################################


# iterate through the list 
for blist in booksList:
     
    # Create a connection to Elasticsearch  

    #Request the Api
    content = requests.get(f"https://api.nytimes.com/svc/books/v3/lists.json?list={blist}&api-key={api_key}")
    #save into the ES DB
    res = content.json()
    pp.pprint(res) 
    docs = res['results']


    """    
    # Prepare documents for indexing
    actions = [
        {
            '_index': index_name,
            '_source': doc
         }
        for doc in docs
    ]



    # Define an index and document type for ES
    index_name = 'NYT-newswire'

    # Bulk index the documents
    headers = {
        'Content-Type': 'application/json'
    }

    index_body = {
        'settings': {
            'number_of_shards': 1,
            'number_of_replicas': 0
        },
        'mappings': {
            'properties': {
                'field1': {'type': 'text'},
                'field2': {'type': 'keyword'}
            }
        }
    }




    #response = es.bulk(index=index_name, body=actions, headers=headers)


  
    # Check the response
    if response['result'] == 'created':
        print(f'{section} saved successfully')
    else:
        print('Failed to save content.')
    
    """
    ######################################################
    time.sleep(1) ##### TO MODIFY ACCORDING API ALLOWANCE
    ######################################################
    
